In [ ]:
import spacy
import pandas as pd

import unidecode 


from spacy.lang.nl import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("yhavinga/t5-v1.1-base-dutch-cnn-test")

model = AutoModelForSeq2SeqLM.from_pretrained("yhavinga/t5-v1.1-base-dutch-cnn-test")

In [ ]:
df=pd.read_csv("data_03.csv",sep="|")

In [ ]:
nlp = spacy.load("nl_core_news_lg")

In [ ]:
#LOWER CASE SEARCH 
#LEMMA THE KEYWORDS
#LEMMA THE ARTICLE

nl_tax_keywords = ["Aanslagjaar","successierechten","Wetboek der Successierechten","fiscale hervorming","fiscale gunstregime","belasting over de toegevoegde waarde",
        "BTW","kadastraal inkomen","onroerende voorheffing","Algemene Administratie van de Fiscaliteit","Inkomstenbelastingen","belastingvermindering","fiscale stimulans",
        "belastbaar tijdperk","Belcotax-on-web","Bedrijfsvoorheffing","Het Wetboek van de inkomstenbelastingen","het Wetboek diverse rechten en taksen",
        "fiscale schuldvorderingen","het Wetboek diverse rechten en taksen","fiscale schuldvorderingen","belastingen op beroepsinkomsten","vennootschapsbelastingen",
        "belastingen op onroerende goederen","verbruiksbelastingen","verkeersbelastingen","afvalbelastingen","provinciebelastingen","milieutaksen","belastingaangifte",
        "belastingteruggave","Biztax","Intervat","Rv-on-web","FinProf","MyMinFinPro","roerende voorheffing","vennootschapsbelasting","rechtspersonenbelasting","belastingkorting", 
        "Bronbelasting", "fiscale bepalingenbelastingontduiking","Belastingontwijking","fiscaal voordeel","Aftrekbare besteding","Belastingvrije som", "Belastingkrediet",
        "Decumul","Forfaitaire beroepskosten","Gemeenschappelijke aanslag","Gemiddelde aanslagvoet","Huwelijksquotiënt","Inkomstenjaar","Marginale aanslagvoet",
        "Onroerende voorheffing","Opcentiemen","Progressieve belastingtarieven","Verbeterd gemiddeld tarief","Vrijstelling met progressievoorbehoud"]


In [55]:
labels=['inkomstenbelasting','Aanslagjaar','Aftrekbarebesteding','afvalbelastingen','Bedrijfsvoorheffing','belastbaartijdperk','belastingaangifte','belastingenopberoepsinkomsten','belastingenoponroerendegoederen','belastingkorting','Belastingkrediet','Belastingontwijking','belastingoverdetoegevoegdewaarde','belastingteruggave','belastingvermindering','Belastingvrijesom','Belcotax-on-web','Biztax','Bronbelasting','BTW','Decumul','FinProf','fiscaalvoordeel','fiscalebepalingenbelastingontduiking','fiscalehervorming','fiscaleschuldvorderingen','fiscaleschuldvorderingen','fiscalestimulans','Forfaitaireberoepskosten','Inkomstenbelastingen','Intervat','kadastraalinkomen','milieutaksen','onroerendevoorheffing','Opcentiemen','Progressievebelastingtarieven','provinciebelastingen','rechtspersonenbelasting','roerendevoorheffing','successierechten','vennootschapsbelasting','verbruiksbelastingen','verkeersbelastingen'
]

In [62]:
nl_tax_list=[]
for _a in labels:
    doc = nlp(_a)
    for x in doc:
        nl_tax_list.append(x.lemma_)



In [ ]:
nl_tax_list

In [57]:
def reducing_incorrect_character_repeatation(text):
    """
    This Function will reduce repeatition to two characters 
    for alphabets and to one character for punctuations.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Finally formatted text with alphabets repeating to 
        two characters & punctuations limited to one repeatition 
        
    Example:
    Input : Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)
    Output : Reallyy, Greeaatt !?.;:)
    
    """
    # Pattern matching for all case alphabets
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    
    # Limiting all the  repeatation to two characters.
    Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
    
    # Pattern matching for all the punctuations that can occur
    Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    
    # Limiting punctuations in previously formatted string to only one.
    Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
    
    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
    return Final_Formatted

In [ ]:
l=df['nl_body']

In [67]:
import re
import string

documents_clean = []
for d in l[14:15]:
    d=re.sub(r'(?<=[.,;,:])(?=[^\s])', r' ', d)
    


    document_test= unidecode.unidecode(d)
    document_test = document_test.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')

    

       
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', document_test)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    document_test = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    document_test = document_test.lower()
    document_test=reducing_incorrect_character_repeatation(document_test)

    document_test = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', document_test) 

   
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    document_test = re.sub(r"[^a-zA-Z:$-,%.?!]+", ' ', document_test)

    documents_clean.append(document_test)
    


In [68]:
text=documents_clean[0]
doc = nlp(text)
type(doc)
len(list(doc.sents))

29

In [71]:
keyword=[]
stopwords = list(STOP_WORDS)
pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
for token in doc:
    if(token.text in stopwords or token.text in punctuation):   # check lemma
        continue
    if(token.pos_ in pos_tag):
        keyword.append(token.lemma_)
freq_word=Counter(keyword)
freq_word.most_common(10)

[('besluit', 21),
 ('koninklijk', 12),
 ('lening', 8),
 ('financiën', 8),
 ('artikel', 7),
 ('minister', 7),
 ('voordeel', 6),
 ('letten', 6),
 ('aan_vullen', 6),
 ('Staatsblad', 6)]

In [72]:
highest={}
for a in nl_tax_list:
    if freq_word.get(a) !=None:
        highest[a]=freq_word.get(a)

highest
    

{'bedrijfsvoorheffing': 1}

In [73]:
doc

federale overheidsdienst financien februari . koninklijk besluit tot wijziging van het kb wib , op het stuk van de voordelen van alle aard in geval van toekenning van een renteloze lening of een lening tegen verminderde rentevoet ( ) filip, koning der belgen, aan allen die nu zijn en hierna wezen zullen, onze groet. gelet op het wetboek van de inkomstenbelastingen , artikel , ss , tweede lid gelet op het kb wib gelet op het advies van de inspecteur van financien, gegeven op januari gelet op het enig artikel van het ministerieel besluit van maart tot toekenning van een bevoegdheidsdelegatie aan de inspectie van financien, waardoor, overeenkomstig artikel van het koninklijk besluit van november betreffende de administratieve en begrotingscontrole, het gunstig advies van de inspecteur van financien geaccrediteerd bij het ministerie van financien onder meer de ontwerpen van koninklijk besluit tot vastlegging van de voordelen van alle aard wat de renteloze leningen of leningen tegen vermind